In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import geopandas as gpd

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec

from playsound import playsound

from scipy.spatial import cKDTree

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append(os.path.abspath(".."))
from function import ART_downscale as ART_down

playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.


## CREATE ENSEMBLE FOR PREVIOUS BIAS CORRECTION CODE

In [2]:
correction, nameout, label = 'Log-Linear-Regression', 'LLc', 'Log-Linear Regression'
# correction, nameout, label = 'Quantile Delta Mapping', 'QDM', 'Quantile Delta Mapping'
# correction, nameout, label = 'Cummulative Distribution', 'CDFt', 'Cummulative Distribution'
# correction, nameout, label = 'Quantile-Quantile', 'QQc', 'Quantile Quantile Mapping'

In [3]:
apply = 'PARAM'
frac = 0.7
seed = 7

In [4]:
REFERENCE = 'CHIRPS' # grid reference
SATELLITES = 'ALL' 

yy_s, yy_e = 2002, 2023
years_num = yy_e - yy_s + 1

npix = 2
acf = 'mar'
cor = 'pearson'

lon_min, lon_max, lat_min, lat_max, area, toll = 6.5, 19, 36.5, 48, 'ITALY', 0.002
Tr = [5,  10,  20,  50, 100, 200]

veneto_dir = os.path.join('/','media','arturo','T9','Data','shapes','Europa','Italy')

if os.path.exists(veneto_dir):
    VENETO = gpd.read_file(os.path.join(veneto_dir,'Veneto.geojson'))
    ITALY = gpd.read_file(os.path.join(veneto_dir,'Italy_clear.geojson'))
else:
    raise SystemExit(f"File not found: {veneto_dir}")

dir_base = os.path.join('/','media','arturo','T9','Data','Italy','Satellite','6_DOWN_BCorrected',apply,nameout)

# CMORPH data
data_dir = os.path.join(dir_base, f'ITALY_DOWN_CMORPH_3h_{yy_s}_{yy_e}_npix_{npix}_thr_1_acf_{acf}_genetic_{cor}_{nameout}_{str(seed).zfill(4)}.nc')
DATA_CM = xr.open_dataset(data_dir)
time_year = DATA_CM.year.values
del DATA_CM

In [5]:
def get_nearest_values(ref_lat2d, ref_lon2d, target_lat2d, target_lon2d, target_data):
    """
    Para cada punto en la malla de referencia, busca el valor más cercano en la malla objetivo.
    """
    ny, nx = ref_lat2d.shape
    ref_points = np.column_stack((ref_lat2d.ravel(), ref_lon2d.ravel()))
    target_points = np.column_stack((target_lat2d.ravel(), target_lon2d.ravel()))
    tree = cKDTree(target_points)
    _, idx = tree.query(ref_points)
    matched_values = target_data.ravel()[idx]
    return matched_values.reshape(ny, nx)

def Create_Ensemble(REFERENCE, PARAM, seed, SATELLITES='ALL'):
    '''
    Create and Ensemble Product using CHIRPS grid
    ALL = ['CMORPH', 'ERA5', 'GSMaP', 'MSWEP', 'IMERG']
    NOGSMaP = ['CMORPH', 'ERA5', 'MSWEP', 'IMERG']
    '''

    data_dir = os.path.join(dir_base, f'ITALY_DOWN_{REFERENCE}_1dy_{yy_s}_{yy_e}_npix_{npix}_thr_1_acf_{acf}_genetic_{cor}_{nameout}_{str(seed).zfill(4)}.nc')
    DATA_REF = xr.open_dataset(data_dir)
    lons_REF, lats_REF = DATA_REF.lon.values, DATA_REF.lat.values
    lon2d_REF, lat2d_REF = np.meshgrid(lons_REF, lats_REF)

    if SATELLITES == 'ALL':

        data_dir = os.path.join(dir_base, f'ITALY_DOWN_CMORPH_3h_{yy_s}_{yy_e}_npix_{npix}_thr_1_acf_{acf}_genetic_{cor}_{nameout}_{str(seed).zfill(4)}.nc')
        DATA_SAT1 = xr.open_dataset(data_dir)
        lon2d_SAT1, lat2d_SAT1 = np.meshgrid(DATA_SAT1.lon.values, DATA_SAT1.lat.values)

        data_dir = os.path.join(dir_base, f'ITALY_DOWN_ERA5_3h_{yy_s}_{yy_e}_npix_{npix}_thr_1_acf_{acf}_genetic_{cor}_{nameout}_{str(seed).zfill(4)}.nc')
        DATA_SAT2 = xr.open_dataset(data_dir)
        lon2d_SAT2, lat2d_SAT2 = np.meshgrid(DATA_SAT2.lon.values, DATA_SAT2.lat.values)

        data_dir = os.path.join(dir_base, f'ITALY_DOWN_GSMaP_3h_{yy_s}_{yy_e}_npix_{npix}_thr_1_acf_{acf}_genetic_{cor}_{nameout}_{str(seed).zfill(4)}.nc')
        # data_dir = os.path.join(dir_base, f'ITALY_DOWN_GSMaP_3h_{yy_s}_{yy_e}_npix_{npix}_thr_1_acf_{acf}_genetic_{cor}_NoCorrection.nc')
        DATA_SAT3 = xr.open_dataset(data_dir)
        lon2d_SAT3, lat2d_SAT3 = np.meshgrid(DATA_SAT3.lon.values, DATA_SAT3.lat.values)

        data_dir = os.path.join(dir_base, f'ITALY_DOWN_MSWEP_3h_{yy_s}_{yy_e}_npix_{npix}_thr_1_acf_{acf}_genetic_{cor}_{nameout}_{str(seed).zfill(4)}.nc')
        DATA_SAT4 = xr.open_dataset(data_dir)
        lon2d_SAT4, lat2d_SAT4 = np.meshgrid(DATA_SAT4.lon.values, DATA_SAT4.lat.values)

        data_dir = os.path.join(dir_base, f'ITALY_DOWN_IMERG_1dy_{yy_s}_{yy_e}_npix_{npix}_thr_1_acf_{acf}_genetic_{cor}_{nameout}_{str(seed).zfill(4)}.nc')
        DATA_SAT5 = xr.open_dataset(data_dir)
        lon2d_SAT5, lat2d_SAT5 = np.meshgrid(DATA_SAT5.lon.values, DATA_SAT5.lat.values)

        DATA_SAT = [DATA_SAT1, DATA_SAT2, DATA_SAT3, DATA_SAT4, DATA_SAT5]
        LATLON_SAT = [
                    (lat2d_SAT1, lon2d_SAT1),
                    (lat2d_SAT2, lon2d_SAT2),
                    (lat2d_SAT3, lon2d_SAT3),
                    (lat2d_SAT4, lon2d_SAT4),
                    (lat2d_SAT5, lon2d_SAT5),]

    else: 
        print('ERROR WITH SATELLITES GROUP')
        return 0

    # ==================================================================================================

    DATA_SAT = [ds.sel(year=DATA_REF.year) for ds in DATA_SAT] 
    ntimes = DATA_REF[PARAM].shape[0] 

    REFE_remap = np.full((ntimes, *lat2d_REF.shape), np.nan)
    SAT_remap = [np.full_like(REFE_remap, np.nan) for _ in range(len(DATA_SAT))]

    for i in range(ntimes):
        REFE = DATA_REF[PARAM].values[i, :, :]
        REFE_remap[i, :, :] = REFE

        for k, (DATA_k, (lat2d_k, lon2d_k)) in enumerate(zip(DATA_SAT, LATLON_SAT)):
            SAT_k = DATA_k[PARAM].values[i, :, :]

            SAT_near = get_nearest_values(lat2d_REF, lon2d_REF, lat2d_k, lon2d_k, SAT_k)

            SAT_remap[k][i, :, :] = SAT_near

    assert all(arr.shape == REFE_remap.shape for arr in SAT_remap)
    stacked_all = np.stack([REFE_remap] + SAT_remap,axis=0)

    # ==================================================================================================
    # Ensemble MEAN and MEDIAN
    Ensemble_mean   = np.nanmean(stacked_all, axis=0)
    Ensemble_median = np.nanmedian(stacked_all, axis=0)

    # Ensemble WEIGHTED
    p = stacked_all.shape[0]
    # 1) median reference (por tiempo o agregado) -> aquí por tiempo
    median = np.nanmedian(stacked_all, axis=0)
    # 2) compute mean absolute dev from median per product (avg over time)
    mad = np.nanmean(np.abs(stacked_all - median), axis=1)  # (p, ny, nx)
    # 3) inverse mad weights (higher weight = closer to median)
    inv_mad = 1.0 / (mad + 1e-6)
    w = inv_mad / np.nansum(inv_mad, axis=0)  # (p, ny, nx)
    # 4) weighted ensemble per time
    Ensemble_weighted = np.nansum(w[:, None, :, :] * stacked_all, axis=0)  # (nt, ny, nx)

    # Ensemble TRIMEAN
    Q25 = np.nanpercentile(stacked_all, 25, axis=0)
    Q50 = np.nanpercentile(stacked_all, 50, axis=0)
    Q75 = np.nanpercentile(stacked_all, 75, axis=0)
    Ensemble_trimean = (Q25 + 2*Q50 + Q75) / 4
    # print()

    return Ensemble_mean, Ensemble_median, Ensemble_weighted, Ensemble_trimean, lons_REF, lats_REF

In [6]:
print(f'Seed: {seed}')
NYd_mean, NYd_median, NYd_weighted, NYd_trimean, lons_REF, lats_REF = Create_Ensemble(REFERENCE, 'NYd', seed, SATELLITES)
CYd_mean, CYd_median, CYd_weighted, CYd_trimean, _, _ = Create_Ensemble(REFERENCE, 'CYd', seed, SATELLITES)
WYd_mean, WYd_median, WYd_weighted, WYd_trimean, _, _ = Create_Ensemble(REFERENCE, 'WYd', seed, SATELLITES)

lon2d_REF, lat2d_REF = np.meshgrid(lons_REF, lats_REF)

Seed: 7


In [7]:
ENSEMBLE_Mevd_mean = ART_down.pre_quantiles_array(
                            NYd_mean, 
                            CYd_mean, 
                            WYd_mean, 
                            Tr, 
                            lats_REF, lons_REF,
                            1)

ENSEMBLE_Mevd_median = ART_down.pre_quantiles_array(
                            NYd_median, 
                            CYd_median, 
                            WYd_median, 
                            Tr, 
                            lats_REF, lons_REF,
                            1)

In [8]:
ENSEMBLE_xr = xr.Dataset(
    data_vars={
        "NYd": (("year","lat","lon"), NYd_mean),
        "CYd": (("year","lat","lon"), CYd_mean),
        "WYd": (("year","lat","lon"), WYd_mean),
        "Mev_d": (("Tr","lat","lon"), ENSEMBLE_Mevd_mean),
    },
    coords={
        'year': time_year, 
        'Tr': Tr, 
        'lat': lats_REF, 
        'lon': lons_REF
    },
    attrs=dict(description=f"ENSEMBLE-mean of the downscaled precipitation data for {yy_s}-{yy_e} period using CHIRPS grid using mean",)
)

ENSEMBLE_xr.NYd.attrs["units"] = "# days"
ENSEMBLE_xr.NYd.attrs["long_name"] = "Corrected Ensemble Downscaled Number of Wet Days"
ENSEMBLE_xr.NYd.attrs["origname"] = "Down Wet Days"

ENSEMBLE_xr.CYd.attrs["units"] = "nondimensional"
ENSEMBLE_xr.CYd.attrs["long_name"] = "Corrected Ensemble Downscaled Scale Parameter"
ENSEMBLE_xr.CYd.attrs["origname"] = "Down Scale"

ENSEMBLE_xr.WYd.attrs["units"] = "nondimensional"
ENSEMBLE_xr.WYd.attrs["long_name"] = "Corrected Ensemble Downscaled Shape Parameter"
ENSEMBLE_xr.WYd.attrs["origname"] = "Down Shape"

ENSEMBLE_xr.Mev_d.attrs["units"] = "mm/day"
ENSEMBLE_xr.Mev_d.attrs["long_name"] = "Corrected Ensemble Downscaled Extreme Quantiles"
ENSEMBLE_xr.Mev_d.attrs["origname"] = "Down Ext-Quant"

ENSEMBLE_xr.lat.attrs["units"] = "degrees_north"
ENSEMBLE_xr.lat.attrs["long_name"] = "Latitude"

ENSEMBLE_xr.lon.attrs["units"] = "degrees_east"
ENSEMBLE_xr.lon.attrs["long_name"] = "Longitude"

PRE_out = os.path.join(os.path.join(dir_base, f'ITALY_ENSEMBLE_{SATELLITES}_1dy_{yy_s}_{yy_e}_npix_{npix}_thr_1_acf_{acf}_genetic_{cor}_mean_{nameout}_{str(seed).zfill(4)}.nc'))
print(f'Export PRE data to {PRE_out}')
ENSEMBLE_xr.to_netcdf(PRE_out)

Export PRE data to /media/arturo/T9/Data/Italy/Satellite/6_DOWN_BCorrected/PARAM/LLc/ITALY_ENSEMBLE_ALL_1dy_2002_2023_npix_2_thr_1_acf_mar_genetic_pearson_mean_LLc_0007.nc


In [9]:
ENSEMBLE_xr = xr.Dataset(
    data_vars={
        "NYd": (("year","lat","lon"), NYd_median),
        "CYd": (("year","lat","lon"), CYd_median),
        "WYd": (("year","lat","lon"), WYd_median),
        "Mev_d": (("Tr","lat","lon"), ENSEMBLE_Mevd_median),
    },
    coords={
        'year': time_year, 
        'Tr': Tr, 
        'lat': lats_REF, 
        'lon': lons_REF
    },
    attrs=dict(description=f"ENSEMBLE of the downscaled precipitation data for {yy_s}-{yy_e} period using CHIRPS grid using median",)
)
ENSEMBLE_xr.NYd.attrs["units"] = "# days"
ENSEMBLE_xr.NYd.attrs["long_name"] = "Corrected Ensemble Downscaled Number of Wet Days"
ENSEMBLE_xr.NYd.attrs["origname"] = "Down Wet Days"

ENSEMBLE_xr.CYd.attrs["units"] = "nondimensional"
ENSEMBLE_xr.CYd.attrs["long_name"] = "Corrected Ensemble Downscaled Scale Parameter"
ENSEMBLE_xr.CYd.attrs["origname"] = "Down Scale"

ENSEMBLE_xr.WYd.attrs["units"] = "nondimensional"
ENSEMBLE_xr.WYd.attrs["long_name"] = "Corrected Ensemble Downscaled Shape Parameter"
ENSEMBLE_xr.WYd.attrs["origname"] = "Down Shape"

ENSEMBLE_xr.Mev_d.attrs["units"] = "mm/day"
ENSEMBLE_xr.Mev_d.attrs["long_name"] = "Corrected Ensemble Downscaled Extreme Quantiles"
ENSEMBLE_xr.Mev_d.attrs["origname"] = "Down Ext-Quant"

ENSEMBLE_xr.lat.attrs["units"] = "degrees_north"
ENSEMBLE_xr.lat.attrs["long_name"] = "Latitude"

ENSEMBLE_xr.lon.attrs["units"] = "degrees_east"
ENSEMBLE_xr.lon.attrs["long_name"] = "Longitude"

PRE_out = os.path.join(os.path.join(dir_base, f'ITALY_ENSEMBLE_{SATELLITES}_1dy_{yy_s}_{yy_e}_npix_{npix}_thr_1_acf_{acf}_genetic_{cor}_median_{nameout}_{str(seed).zfill(4)}.nc'))
print(f'Export PRE data to {PRE_out}')
ENSEMBLE_xr.to_netcdf(PRE_out)

Export PRE data to /media/arturo/T9/Data/Italy/Satellite/6_DOWN_BCorrected/PARAM/LLc/ITALY_ENSEMBLE_ALL_1dy_2002_2023_npix_2_thr_1_acf_mar_genetic_pearson_median_LLc_0007.nc


In [10]:
playsound("../sound/HOMER_DOH.mp3")